## Geographical plots

In this part of the hands-on we will be plotting information on geographical. This goes by the name of Geographic Information Systems (GIS), which, as the name implies, is some sort of information visualization.

GIS comes in many forms but for this example we will be making a **Choropleth**. A Choropleth is a map visualisation where parts of a map are shaded in proportion to a data measurement or statistical value of choice. 

For this example we are going to visualise the results of the general election of 2023 for each municipality in the Netherlands. For this we will first need to import Pandas and prepare our data.

In [1]:
%reset -f

import pandas

# Load the data with a semi-colon delimiter which is used in the csv file
results = pandas.read_csv('./data/geographical/TK2023_results.csv', delimiter=';')

# Reformat the dataframe so that every political party is a separate column and every row is a unique municipality entry 
results = results.groupby(['municipality','type'])['value'].sum().unstack(level=1).reset_index()

# Create a party columns only dataframe
party_df = results.drop(columns=['municipality', 'AantalBlancoStemmen', 'AantalGeldigeStemmen', 'AantalOngeldigeStemmen', 'Kiesgerechtigden', 'Opkomst'])

# Change the number of votes for a party to the total percentage voted for that party
party_information = {f'p_{party_name}': 100.0 * party_df[party_name] / results['AantalGeldigeStemmen'] for party_name in party_df.columns}

# All columns with the general voter information
voters_information = {'municipality': results['municipality'], 
                      'p_turnout': 100.0 * results['Opkomst'] / results['Kiesgerechtigden'],
                      'p_valid': 100.0 * results['AantalGeldigeStemmen'] / results['Opkomst'],
                      'p_invalid': 100.0 * results['AantalOngeldigeStemmen'] / results['Opkomst'],
                      'p_blank': 100.0 * results['AantalBlancoStemmen'] / results['AantalGeldigeStemmen']}

# Combine all the data columns
p_results = pandas.DataFrame(dict(voters_information, **party_information))

# Replace all NaN with zero
p_results = p_results.fillna(0) 

p_results

,municipality,p_turnout,p_valid,p_invalid,p_blank,p_50PLUS,p_BBB,p_BIJ1,p_BVNL / Groep Van Haga,p_CDA,...,p_Partij voor de Dieren,p_PartijvdSport,p_Piratenpartij - De Groenen,p_Politieke Partij voor Basisinkomen,p_SP (Socialistische Partij),p_Samen voor Nederland,p_Splinter,p_Staatkundig Gereformeerde Partij (SGP),p_VVD,p_Volt
0,'s-Gravenhage,68.799740,99.352523,0.419864,0.229096,0.381281,1.287438,0.827200,0.405418,1.917861,...,3.387348,0.000000,0.116594,0.031092,2.641559,0.042137,0.146867,0.378418,15.115428,2.275005
1,'s-Hertogenbosch,78.048583,99.620574,0.183996,0.196175,0.523828,2.002442,0.350610,0.429915,2.587835,...,2.458443,0.038609,0.092870,0.013565,4.229232,0.042783,0.125218,0.117913,17.569105,2.394791
2,Aa en Hunze,85.084762,99.698627,0.117201,0.184729,0.335871,9.577922,0.117555,0.475817,2.485446,...,2.507837,0.000000,0.089566,0.000000,3.493059,0.083968,0.078370,0.240708,16.222571,1.371473
3,Aalsmeer,82.894236,99.668401,0.178553,0.153555,0.588627,3.383324,0.261043,0.578390,3.695552,...,1.648155,0.071659,0.071659,0.000000,1.991094,0.030711,0.194503,0.644930,25.070379,1.330808
4,Aalten,82.456873,99.803161,0.089984,0.107066,0.349375,15.575341,0.101431,0.569142,7.150907,...,1.200270,0.000000,0.067621,0.000000,2.744280,0.236673,0.101431,0.642398,11.574439,0.766370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,Zundert,80.012650,99.698167,0.114984,0.187414,0.634326,10.898868,0.100915,0.598284,3.928494,...,1.290276,0.043249,0.028833,0.014416,3.481583,0.028833,0.108124,0.100915,20.471419,1.066820
339,Zutphen,79.927054,99.633653,0.157465,0.209650,0.358018,4.938073,0.229003,0.432202,2.299703,...,4.186557,0.000000,0.103212,0.000000,4.880015,0.067733,0.116114,0.351568,11.759773,1.793317
340,Zwartewaterland,87.726159,99.813718,0.119753,0.066653,0.226621,7.978404,0.033327,0.426581,4.992335,...,0.733187,0.013331,0.033327,0.000000,1.626341,0.019996,0.013331,20.662534,7.631807,0.479904
341,Zwijndrecht,73.288049,99.588070,0.217848,0.194885,0.628406,2.282942,0.262499,0.342044,3.913614,...,1.571014,0.083522,0.059659,0.000000,2.986915,0.083522,0.119318,4.271567,14.851052,0.727837


As you can see we now prepared our data and created a DataFrame that consists of the blank votes, invalid votes, valid votes, turnout and all parties in percentages. This way we can easily color code the municipalities and compare them to each other based on the data columns in the DataFrame. 

Now lets create the map. The first map we will create will be a choropleth displaying the 2023 general elections turnout per municipality.

In [2]:
from IPython.display import display, IFrame
import folium

geo_path = r'./data/geographical/gemeentegrenzen.geojson'

ref_map = folium.Map(
    location=[52.239177, 5.327108], # This will center the view on the world map where the Netherlands is located
    tiles="cartodbpositron",          # This creates a base map from  which is less distracting than the default Openstreetmap map
    zoom_start=8)                   # This will zoom in on the center of view to get the Netherlands in full frame

folium.Choropleth(
    # This is the path to the geojson file that contains all the municipality shapes and locations             
    geo_data=geo_path,                     
    # We will use the percentage dataframe for the choropleth mapping          
    data=p_results,                        
    # Municipality will be used for the mapping key and p_turnout for its value
    columns=['municipality', 'p_turnout'], 
    # Use statnaam (this is the key in the geojson file that has the municipality name) as keys for colormapping 
    key_on='feature.properties.statnaam',   
    # We are going to use a color map from yellow to green
    fill_color='YlGn',                     
    # This gives municipality shapes some opacity so that we can still see the background
    fill_opacity=0.7,                      
    # This gives the lines around the municipality shapes some opacity so that they don't stand out too much
    line_opacity=0.2,                      
    # The legend
    legend_name='Turn out (%)').add_to(ref_map)            

ref_map

If everything went according to plan you should see a map of the Netherlands with patches shaded green propotional to the *p_turnout* data values. From this choropleth you can already see that the percentage of turn out is relatively lower in and around the bigger cities.

* * *
* * *

#### Exercise

For this exercise you are going to look at how the other data columns are represented across the Netherlands by changing the **columns** variable second entry, in the **folium.Choropleth** function below, to one of the other data columns in our DataFrame. Also try to represent the results with the right color map (in personal preferences of course), from purple to blue (PuBu), from yellow to green (YlGn) or from orange to red (OrRd).

In [3]:
colorMaps = ['PuBu', 'YlGn', 'OrRd']

geo_path = r'./data/geographical/gemeentegrenzen.geojson'

ref_map = folium.Map(
    location=[52.139177, 5.327108], # This will center the view on the world map where the Netherlands is located
    tiles="cartodbpositron",          # This creates a base map from  which is less distracting than the default Openstreetmap map
    zoom_start=8)                   # This will zoom in on the center of view to get the Netherlands in full frame

folium.Choropleth(
    # This is the path to the geojson file that contains all the municipality shapes and locations      
    geo_data=geo_path,                     
    # We will use the percentage dataframe for the choropleth mapping      
    data=p_results,                       
    # Municipality will be used for the mapping key and one of the other datacolumns as a value     
    columns=['municipality', 'p_50PLUS'],  
    # Use statnaam as keys for colormapping   
    key_on='feature.properties.statnaam',   
    # We are going to use a color map stored in the colormap dictionary
    fill_color=colorMaps[2],      
    # This gives municipality shapes some opacity so that we can still see the background       
    fill_opacity=0.7,                      
    # This gives the lines around the municipality shapes some opacity so that they don't stand out too much     
    line_opacity=0.2,                 
    # The legend
    legend_name='p_50PLUS description').add_to(ref_map)

ref_map

***
***

This chapter concludes the main visualation chapters of this course now you may proceed the conclusion notebook of this course for some take away messages: [Conclusion](./5.&#32;Conclusion.ipynb)